### Imports

In [ ]:
import json
import os
from typing import List

from dotenv import load_dotenv
from IPython.display import Image, display
from json_repair import repair_json
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import ChatOllama
from langgraph.graph import END, START, StateGraph
from typing_extensions import TypedDict

In [ ]:
load_dotenv()
ollama_base_url = ""

if os.getenv("BASE_URL"):
    ollama_base_url = os.getenv("BASE_URL")
else:
    ollama_base_url = None

### LLM Definitions

In [ ]:
load_dotenv()

relations_extractor_llm = "llama3.1"
relations_json_formatter_llm = "llama3.1"
summary_llm = "llama3.1"
summary_json_formatter_llm = "llama3.1"
name_date_llm = "llama3.1"
name_date_json_formatter_llm = "llama3.1"

### Relationship Extractor

In [ ]:
llm = ChatOllama(
    base_url=ollama_base_url,
    model=relations_extractor_llm,
    format="json",
    temperature=0,
)

prompt = PromptTemplate(
    template="""
     You are an expert in analyzing financial circulars and extracting key information. Your task is to carefully read the given PDF content from a circular and extract specific details in a structured JSON format. Start extracting relationships after the Title of the document.

    Relationships in the context is defined as mention of a particular document, circular, notification, laws, acts. in the current document. Extract the most meaningful single word relationship mentioned in the document. While mentioning the document, please only include the title of the document, date and other identifiers are not required. 
    
    Example of relationships include but are not limited to: superseded, amended, overturned, replaced, etc.

    <JSON Output Structure>
    "relations": {{
      "Document 1": "Relationship with current document",
      "Document 2": "Relationship with current document"
    }}
    </JSON Structure>

    <PDF Page Content>
    {pdf_content}
    </PDF Page Content>
    """,
    input_variables=["pdf_content"],
)

page_relations_extractor_chain = prompt | llm | JsonOutputParser()

In [ ]:
llm = ChatOllama(
    base_url=ollama_base_url,
    model=relations_json_formatter_llm,
    format="json",
    temperature=0,
)

prompt = PromptTemplate(
    template="""
    Given the Required Output JSON schema, you are required to format by combining individual string output into a single structured JSON format. Remove or Merge any duplicate or redundant information.

    <Unformatted Input>
    {unformatted_input}
    </Unformatted Input>

    <Formatted JSON Output Schema>
    "relations": {{
      "Document 1": "Relationship with current document",
      "Document 2": "Relationship with current document"
    }}
    </Formatted JSON Output Schema>

    """,
    input_variables=["pdf_content"],
)

relations_json_formatter_chain = prompt | llm | JsonOutputParser()

In [ ]:
# pdf = PyPDFLoader('/workspace/legalAgent/langraphKG/data/circulars/Existing Circular.pdf').load()
# output = ""

# for i in range(len(pdf)):
#     page_output = page_relations_extractor_chain.invoke({'pdf_content': pdf[i].page_content})
#     page_output = str(page_output)
#     output += page_output + "\n"

# output = relations_json_formatter_chain.invoke({'unformatted_input': output})
# output = repair_json(str(output))
# print(output)

### Summary Generator

In [ ]:
llm = ChatOllama(
    base_url=ollama_base_url,
    model=summary_llm,
    format="json",
    temperature=0,
)

prompt = PromptTemplate(
    template="""
    You are an expert in analyzing financial circulars and extracting key information. Your task is to carefully read the given PDF content from a circular and extract specific details in a structured JSON format. 

    Summarize the content of the document. The summary should be concise and capture the essence of the document.
    

    <JSON Output Structure>
    "summary": "summary of the document"
    </JSON Structure>

    <PDF Page Content>
    {pdf_content}
    </PDF Page Content>
    """,
    input_variables=["pdf_content"],
)

summary_chain = prompt | llm | JsonOutputParser()

In [ ]:
llm = ChatOllama(
    base_url=ollama_base_url,
    model=summary_json_formatter_llm,
    format="json",
    temperature=0,
)

prompt = PromptTemplate(
    template="""
    Given the Required Output JSON schema, you are required to format by using the page wise summary to generate a single complete summary for the document and format it in JSON format. Remove or Merge any duplicate or redundant information.

    <Unformatted Input>
    {unformatted_input}
    </Unformatted Input>

    <Formatted JSON Output Schema>
    "summary": "summary of the document"
    </Formatted JSON Output Schema>
    """,
    input_variables=["pdf_content"],
)

summary_json_formatter_chain = prompt | llm | JsonOutputParser()

In [ ]:
# pdf = PyPDFLoader('/workspace/fmc/agenticKG/langraphApproach/Existing Circular.pdf').load()
# output = ""

# for i in range(len(pdf)):
#     page_output = summary_chain.invoke({'pdf_content': pdf[i].page_content})
#     page_output = str(page_output)
#     output += page_output + "\n"

# output = summary_json_formatter_chain.invoke({'unformatted_input': output})
# output = repair_json(str(output))
# print(output)

### Name and Date Extractor

In [ ]:
llm = ChatOllama(
    base_url=ollama_base_url,
    model=name_date_llm,
    format="json",
    temperature=0,
)

prompt = PromptTemplate(
    template="""
    You are an expert in analyzing financial circulars and extracting key information. Your task is to carefully read the given PDF content from a circular and extract specific details in a structured JSON format. 

    1. Name: Provide the name of the document mentioned at the very start not including RBI/Year/No
    2. Date of Issue: Find and extract the date when the circular was issued.
    
    <JSON Output Structure>
    "name": "The full name of the document without '/'",
    "date_of_issue": "The date when the circular was issued in DD/MM/YYYY format",
    </JSON Structure>

    <PDF Page Content>
    {pdf_content}
    </PDF Page Content>
    """,
    input_variables=["pdf_content"],
)

name_date_chain = prompt | llm | JsonOutputParser()

In [ ]:
# pdf = PyPDFLoader('/workspace/fmc/agenticKG/langraphApproach/Existing Circular.pdf').load()
# output = name_date_chain.invoke({'pdf_content': pdf[0].page_content})
# print(output)

### State Definition

In [ ]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        path: The document path.
        name: The name of the document.
        date_of_issue: The date when the circular was issued.
        summary: The summary of the document.
        relations: The relationships between the current document and other documents.
    """

    path: str
    name: str
    date_of_issue: str
    summary: str
    relations: List[str]

### Node Definitions

In [ ]:
def extract_name_date(state):
    """
    Extract name and date of issue

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New keys added to state, name and date_of_issue, that contain the name and date of issue
    """
    print("---EXTRACTING NAME AND DATE---")
    pdf = PyPDFLoader(state["path"]).load()
    output = name_date_chain.invoke({"pdf_content": pdf[0].page_content})
    state["name"] = output["name"].replace("/", " ")
    state["date_of_issue"] = output["date_of_issue"]
    return state


def extract_relationships(state):
    """
    Extract relationships

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, relations, that contains the relationships between the current document and other documents
    """
    print("---EXTRACTING RELATIONSHIPS---")
    pdf = PyPDFLoader(state["path"]).load()
    output = ""
    for i in range(len(pdf)):
        page_output = page_relations_extractor_chain.invoke(
            {"pdf_content": pdf[i].page_content}
        )
        page_output = str(page_output)
        output += page_output + "\n"
    output = relations_json_formatter_chain.invoke({"unformatted_input": output})
    output = repair_json(str(output))
    output = json.loads(output)
    state["relations"] = output["relations"]
    return state


def summarize_document(state):
    """
    Summarize the document

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, summary, that contains the summary of the document
    """
    print("---SUMMARIZING DOCUMENT---")
    pdf = PyPDFLoader(state["path"]).load()
    output = ""
    for i in range(len(pdf)):
        page_output = summary_chain.invoke({"pdf_content": pdf[i].page_content})
        page_output = str(page_output)
        output += page_output + "\n"
    output = summary_json_formatter_chain.invoke({"unformatted_input": output})
    output = repair_json(str(output))
    output = json.loads(output)
    state["summary"] = output["summary"]
    return state


def format_output(state):
    """
    Format the output

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): The formatted output
    """
    print("---FORMATTING OUTPUT---")
    output = {
        "name": state["name"],
        "date_of_issue": state["date_of_issue"],
        "summary": state["summary"],
        "relations": state["relations"],
    }
    output = repair_json(str(output))
    output = json.loads(output)

    output_path = "./output/"
    os.makedirs(output_path, exist_ok=True)
    with open(f"{output_path}{state['name']}.json", "w") as f:
        json.dump(output, f)

    return state

### Graph Building

In [ ]:
workflow = StateGraph(GraphState)

workflow.add_node("name_date", extract_name_date)
workflow.add_node("relationships", extract_relationships)
workflow.add_node("doc_summary", summarize_document)
workflow.add_node("output_formatting", format_output)

workflow.add_edge(START, "name_date")
workflow.add_edge("name_date", "relationships")
workflow.add_edge("relationships", "doc_summary")
workflow.add_edge("doc_summary", "output_formatting")
workflow.add_edge("output_formatting", END)

app = workflow.compile()

In [ ]:
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

### Executing Graph

In [ ]:
directory_path = "circulars2/"
for filename in os.listdir(directory_path):
    print(f"Processing {filename}...")
    inputs = {
        "path": f"{directory_path}{filename}",
        "name": "",
        "date_of_issue": "",
        "summary": "",
        "relations": [],
    }
    for output in app.stream(inputs):
        for key, value in output.items():
            print("---" * 5)
            print(f"{key}: {value}")
            print("---" * 5)